In [1]:
import pandas as pd

# Interpolated data

In [2]:
data = pd.read_csv("Data/Interpolated_soil_combined/combined_zones.csv")
data = data[data['PhosphorusTreatment'] != 0]
data = data[data['Year'] == 2018]
print(data.shape)
data.head()

(1411, 71)


,Longitude,Latitude,Yield,Moisture,PlotID,FieldID,elevation,slope,aspect,TOC_0_2,...,Ols_Fe_2_6,Ols_K_2_6,Ols_Mg_2_6,Ols_P_2_6,Zone_1,Zone_2,Zone_3,Zone_4,Year,PhosphorusTreatment
6,-96.893077,31.483352,32.446,11.5,3,16A,166.270996,1.823310,285.239014,182.445999,...,1.51234,178.923996,49.166801,3.43851,1,0,0,0,2018,11.3
7,-96.893106,31.483338,28.649,11.5,3,16A,166.182999,0.652037,296.753998,182.692001,...,1.51228,178.764999,49.349899,3.45035,1,0,0,0,2018,11.3
34,-96.892190,31.483776,41.226,11.5,3,16A,167.481995,0.236201,206.576996,162.488998,...,1.38079,195.936005,44.266899,3.20947,1,0,0,0,2018,11.3
35,-96.892218,31.483763,41.130,11.5,3,16A,167.459000,0.269463,130.031998,162.171997,...,1.38109,195.445999,44.655499,3.21152,1,0,0,0,2018,11.3
36,-96.892245,31.483750,42.928,11.5,3,16A,167.486999,1.284780,195.367004,164.294998,...,1.51713,194.266998,45.267300,3.20237,1,0,0,0,2018,11.3


In [3]:
data.columns.values

array(['Longitude', 'Latitude', 'Yield', 'Moisture', 'PlotID', 'FieldID',
       'elevation', 'slope', 'aspect', 'TOC_0_2', 'TC_0_2', 'IC_0_2',
       'TN_0_2', 'DI_Al_0_2', 'DI_Ca_0_2', 'DI_Fe_0_2', 'DI_K_0_2',
       'DI_Mg_0_2', 'DI_P_0_2', 'H3A_Al_0_2', 'H3A_Ca_0_2', 'H3A_Fe_0_2',
       'H3A_K_0_2', 'H3A_Mg_0_2', 'H3A_P_0_2', 'M3_Al_0_2', 'M3_Ca_0_2',
       'M3_Fe_0_2', 'M3_K_0_2', 'M3_Mg_0_2', 'M3_P_0_2', 'Ols_Al_0_2',
       'Ols_Ca_0_2', 'Ols_Fe_0_2', 'Ols_K_0_2', 'Ols_Mg_0_2', 'Ols_P_0_2',
       'TOC_2_6', 'TC_2_6', 'IC_2_6', 'TN_2_6', 'DI_Al_2_6', 'DI_Ca_2_6',
       'DI_Fe_2_6', 'DI_K_2_6', 'DI_Mg_2_6', 'DI_P_2_6', 'H3A_Al_2_6',
       'H3A_Ca_2_6', 'H3A_Fe_2_6', 'H3A_K_2_6', 'H3A_Mg_2_6', 'H3A_P_2_6',
       'M3_Al_2_6', 'M3_Ca_2_6', 'M3_Fe_2_6', 'M3_K_2_6', 'M3_Mg_2_6',
       'M3_P_2_6', 'Ols_Al_2_6', 'Ols_Ca_2_6', 'Ols_Fe_2_6', 'Ols_K_2_6',
       'Ols_Mg_2_6', 'Ols_P_2_6', 'Zone_1', 'Zone_2', 'Zone_3', 'Zone_4',
       'Year', 'PhosphorusTreatment'], dtype=object)

In [4]:
def convert_to_cm(inch_str):
    # Convert inches to cm (1 inch = 2.54 cm)
    return str(round(int(inch_str) * 2.54))

def process_column_name(column_name):
    # Split the string by underscores
    parts = column_name.split('_')
    
    # Convert the last two numbers to cm
    parts[-2] = convert_to_cm(parts[-2])
    parts[-1] = convert_to_cm(parts[-1])
    
    # Reassemble the string, replacing underscores with spaces
    # and putting the last two numbers in parentheses with a hyphen between them
    return ' '.join(parts[:-2]) + f" ({parts[-2]}-{parts[-1]})"

In [5]:
columns = ['TOC_0_2', 'TOC_2_6', 'IC_0_2', 'IC_2_6', 'TN_0_2', 'TN_2_6', 
           'DI_P_0_2', 'DI_P_2_6', 'H3A_P_0_2', 'H3A_P_2_6', 'M3_P_0_2', 'M3_P_2_6', 'Ols_P_0_2', 'Ols_P_2_6']

new_columns = [process_column_name(column) for column in columns]

table_data = data.groupby(['FieldID']).mean()[columns]
table_data_std = data.groupby(['FieldID']).std()[columns]
# Calculate mean and std
means = table_data.round(1)
stds = table_data_std.round(1)

# Create new DataFrame to hold values
results = means.copy()

# Loop through columns
for i, col in enumerate(means.columns):
  values = []
  for j in range(len(means)):
      # Format string with mean and std  
      fmt_str = '{0} ({1})'  
      values.append(fmt_str.format(means.iloc[j, i], stds.iloc[j, i]))
      # Add to dataframe
  results[col] = values

# Set index
# results.index = [1,2,3,4,5]
results.index.name = 'Field'
results['Soil test'] = ['Mehlich-3', 'Mehlich-3', 'Mehlich-3', 'Haney', 'Haney']
results['P recommendation'] = [22.5, 22.5, 18, 11.3, 6.8]
# Converting to metric
results['P recommendation'] = round(results['P recommendation']*67.25, 1)
results = results[['Soil test', 'P recommendation'] + columns].T
results.index = ['Soil test', 'P recommendation'] + new_columns
# results.to_csv('../../Manuscript/Tables/Table 1.csv')
results

Field,16A,6-12,SW16,Y10,Y8
Soil test,Mehlich-3,Mehlich-3,Mehlich-3,Haney,Haney
P recommendation,1513.1,1513.1,1210.5,759.9,457.3
TOC (0-5),170.3 (11.7),230.9 (10.8),214.9 (4.6),208.0 (4.1),350.5 (6.6)
TOC (5-15),146.6 (4.9),214.5 (8.8),203.7 (8.8),191.2 (3.5),344.6 (6.3)
IC (0-5),46.8 (10.4),94.6 (5.2),70.3 (7.5),52.1 (6.4),104.9 (14.1)
IC (5-15),58.9 (10.9),108.7 (5.1),80.5 (7.3),68.2 (8.9),120.9 (14.3)
TN (0-5),18.3 (1.5),77.3 (45.2),38.9 (0.7),29.4 (2.2),50.1 (2.2)
TN (5-15),17.7 (1.9),41.4 (4.5),41.9 (1.0),24.7 (1.8),49.7 (1.9)
DI P (0-5),30.3 (14.9),3.5 (0.5),3.1 (0.1),18.1 (1.7),23.5 (2.3)
DI P (5-15),21.2 (11.6),3.6 (0.4),3.3 (0.2),13.7 (2.1),21.6 (3.1)


# Raw data

In [6]:
# Path to the Excel file
file_path = 'Data/Soil_data/2018 Fertility yield and soils for input.xlsx'

# Tabs you want to read
tabs = ['16A', '6-12', 'Y10', 'Y8', 'SW16']

# Depths you're interested in
depths = ['0-2', '2-6']

# Columns you're interested in
cols = ['PlotID', 'Depth', 'TOC', 'TC', 'IC', 'TN', 'DI Al', 'DI As', 'DI Ca', 'DI Fe', 'DI K',
       'DI Mg', 'DI Mn', 'DI P', 'DI S', 'DI Zn', 'DI NO3N', 'DI SRP',
       'DI NH4N ', 'H3A Al', 'H3A As', 'H3A Ca', 'H3A Fe', 'H3A K', 'H3A Mg',
       'H3A Mn', 'H3A P', 'H3A S', 'H3A Zn', 'H3A NO3', 'H3A P color',
       'H3A NH4N', 'M3 Al', 'M3 As', 'M3 Ca', 'M3 Fe', 'M3 K', 'M3 Mg',
       'M3 Mn', 'M3 P', 'M3 S', 'M3 Zn', 'Ols Al', 'Ols As', 'Ols Ca',
       'Ols Fe', 'Ols K', 'Ols Mg', 'Ols Mn', 'Ols P', 'Ols S', 'Ols Zn']

# Initialize an empty list to store DataFrames
final_dfs = []

for tab in tabs:
    dfs_by_depth = []
    for depth in depths:
        # Read data from the current tab
        df = pd.read_excel(file_path, sheet_name=tab, usecols=cols, engine='openpyxl')
        
        # Filter the data by PlotID and depth
        df_filtered = df.loc[(df['PlotID'] > 0) & (df['Depth'] == depth)].copy()
        
        # Convert PlotID to string type
        df_filtered.loc[:, 'PlotID'] = df_filtered['PlotID'].astype(int).astype(str)
        
        # Rename columns to append the depth information
        df_filtered.rename(columns={col: f"{col}_{depth}" for col in cols if col != 'PlotID'}, inplace=True)
        
        # Drop the Depth column as it's now represented in the column headers
        df_filtered.drop(columns=[f"Depth_{depth}"], inplace=True)
        
        dfs_by_depth.append(df_filtered)
    
    # Merge DataFrames by 'PlotID'
    merged_df = dfs_by_depth[0]
    for df in dfs_by_depth[1:]:
        merged_df = pd.merge(merged_df, df, on='PlotID', how='outer')
    
    # Add a new column for FieldID based on the tab name
    merged_df['FieldID'] = tab
    
    final_dfs.append(merged_df)

# Concatenate all DataFrames in the list into a single DataFrame
final_df = pd.concat(final_dfs, ignore_index=True)

# Replace hyphens and spaces with underscores in column headers
final_df.columns = final_df.columns.str.replace('-', '_').str.replace(' ', '_')

In [9]:
final_df

,PlotID,TOC_0_2,TC_0_2,IC_0_2,TN_0_2,DI_Al_0_2,DI_As_0_2,DI_Ca_0_2,DI_Fe_0_2,DI_K_0_2,...,Ols_As_2_6,Ols_Ca_2_6,Ols_Fe_2_6,Ols_K_2_6,Ols_Mg_2_6,Ols_Mn_2_6,Ols_P_2_6,Ols_S_2_6,Ols_Zn_2_6,FieldID
0,21,206.208070,243.806134,37.598064,17.880394,514.0,0.36,2160.0,94.0,224.0,...,0.27,192.0,1.40,169.0,73.3,0.20,5.44,13.10,0.00,16A
1,22,166.990408,232.604772,65.614364,28.281983,464.0,0.46,3360.0,46.6,364.0,...,0.21,95.8,1.41,214.0,32.8,0.09,3.06,11.70,0.04,16A
2,19,210.563461,236.414828,25.851368,21.066676,526.0,0.32,2160.0,94.0,212.0,...,0.30,139.0,1.65,166.0,62.4,0.19,4.36,13.30,0.02,16A
3,20,184.024273,254.881196,70.856924,15.842852,496.0,0.42,3220.0,45.2,370.0,...,0.26,106.0,1.32,203.0,31.1,0.09,3.56,13.00,0.00,16A
4,17,205.080035,234.835405,29.755370,23.830628,536.0,0.30,2160.0,96.2,206.0,...,0.02,112.0,1.12,168.0,64.3,0.13,4.30,13.40,0.02,16A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,11,216.567575,298.564143,81.996568,35.956772,319.0,0.00,193.0,181.0,45.0,...,0.00,85.1,1.09,179.0,46.6,0.08,3.80,13.30,0.04,SW16
114,13,215.752869,278.388766,62.635897,29.588834,271.0,0.07,186.0,132.0,34.2,...,0.00,74.1,0.30,188.0,73.8,0.09,3.86,14.20,0.01,SW16
115,14,224.445397,303.341520,78.896123,42.721784,269.0,0.04,193.0,153.0,41.1,...,0.10,89.8,1.14,171.0,39.9,0.10,3.75,13.50,0.00,SW16
116,16,245.642046,312.014582,66.372536,51.785388,247.0,0.05,201.0,120.0,37.2,...,0.00,22.4,0.00,50.9,20.5,0.03,1.65,4.92,0.00,SW16


In [16]:
columns = ['TOC_0_2', 'TOC_2_6', 'IC_0_2', 'IC_2_6', 'TN_0_2', 'TN_2_6', 
           'DI_P_0_2', 'DI_P_2_6', 'H3A_P_0_2', 'H3A_P_2_6', 'M3_P_0_2', 'M3_P_2_6', 'Ols_P_0_2', 'Ols_P_2_6']

final_df_filtered = final_df[['FieldID'] + columns]

new_columns = [process_column_name(column) for column in columns]

table_data = final_df_filtered.groupby(['FieldID']).mean()[columns]
table_data_std = final_df_filtered.groupby(['FieldID']).std()[columns]
# Calculate mean and std
means = table_data.round(1)
stds = table_data_std.round(1)

# Create new DataFrame to hold values
results = means.copy()

# Loop through columns
for i, col in enumerate(means.columns):
  values = []
  for j in range(len(means)):
      # Format string with mean and std  
      fmt_str = '{0} ({1})'  
      values.append(fmt_str.format(means.iloc[j, i], stds.iloc[j, i]))
      # Add to dataframe
  results[col] = values

# Set index
# results.index = [1,2,3,4,5]
results.index.name = 'Field'
results['Basis of fertilizer rec.'] = ['Mehlich-3', 'Mehlich-3', 'Mehlich-3', 'Haney', 'Haney']
results['P recommendation'] = [22.5, 22.5, 18, 11.3, 6.8]
# Converting to metric
results['Rec. fertilizer P rate'] = round(results['P recommendation']*0.45359/0.0404686, 1)
results['Relative fertility'] = ['Low P', 'Low P', 'Low P', 'High P', 'High P']
results = results[['Relative fertility', 'Basis of fertilizer rec.', 'Rec. fertilizer P rate'] + columns].T
results.index = ['Relative fertility', 'Basis of fertilizer rec.', 'Rec. fertilizer P rate'] + new_columns
results['Units'] = ['', '', 'kg ha-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1', 'mg kg-1']
results = results[['Units', '16A', '6-12', 'SW16', 'Y10', 'Y8']]
results.to_csv('../../Manuscript/Tables/Table 1 raw.csv')
results

Field,Units,16A,6-12,SW16,Y10,Y8
Relative fertility,,Low P,Low P,Low P,High P,High P
Basis of fertilizer rec.,kg ha-1,Mehlich-3,Mehlich-3,Mehlich-3,Haney,Haney
Rec. fertlizer P rate,,252.2,252.2,201.8,126.7,76.2
TOC (0-5),mg kg-1,168.6 (23.0),230.5 (31.1),217.8 (25.5),209.5 (35.0),352.1 (41.3)
TOC (5-15),mg kg-1,147.0 (19.9),213.3 (29.5),207.5 (21.0),191.7 (17.5),348.2 (46.3)
IC (0-5),mg kg-1,48.3 (13.1),95.3 (13.3),72.5 (9.5),52.4 (10.3),105.3 (21.8)
IC (5-15),mg kg-1,60.8 (14.3),108.7 (14.8),81.2 (10.7),68.4 (12.8),121.2 (22.3)
TN (0-5),mg kg-1,18.7 (4.4),79.7 (197.1),39.4 (6.6),29.2 (5.9),50.0 (7.6)
TN (5-15),mg kg-1,17.7 (4.0),40.2 (23.2),42.4 (7.3),24.7 (4.1),50.2 (7.9)
DI P (0-5),mg kg-1,26.4 (47.8),3.6 (1.0),3.1 (0.5),18.1 (5.1),23.7 (6.2)


In [9]:
columns = ['M3_Ca_0_2', 'M3_Ca_2_6', 'H3A_Ca_0_2', 'H3A_Ca_2_6', 'Ols_Ca_0_2', 'Ols_Ca_2_6', 'DI_Ca_0_2', 'DI_Ca_2_6']

final_df_filtered = final_df[['FieldID'] + columns]

new_columns = [process_column_name(column) for column in columns]

table_data = final_df_filtered.groupby(['FieldID']).mean()[columns]
table_data_std = final_df_filtered.groupby(['FieldID']).std()[columns]
# Calculate mean and std
means = table_data.round(1)
stds = table_data_std.round(1)

# Create new DataFrame to hold values
results = means.copy()

# Loop through columns
for i, col in enumerate(means.columns):
  values = []
  for j in range(len(means)):
      # Format string with mean and std  
      fmt_str = '{0} ({1})'  
      values.append(fmt_str.format(means.iloc[j, i], stds.iloc[j, i]))
      # Add to dataframe
  results[col] = values

results.T

FieldID,16A,6-12,SW16,Y10,Y8
M3_Ca_0_2,2980.0 (505.4),4163.8 (176.9),3591.1 (217.7),3442.7 (254.7),3975.3 (84.8)
M3_Ca_2_6,2919.7 (791.0),4143.8 (181.2),3585.6 (204.7),3536.4 (260.4),4006.6 (103.8)
H3A_Ca_0_2,2543.5 (779.3),2016.2 (163.2),2001.7 (83.9),1650.5 (211.6),2089.4 (64.8)
H3A_Ca_2_6,2547.1 (788.9),2030.0 (152.9),2026.7 (76.8),1615.2 (398.3),2121.2 (74.6)
Ols_Ca_0_2,117.9 (41.2),45.5 (8.7),75.5 (23.0),174.9 (34.4),104.4 (31.1)
Ols_Ca_2_6,101.8 (25.4),36.5 (7.9),67.4 (25.7),152.7 (26.2),91.6 (33.7)
DI_Ca_0_2,2855.5 (457.6),223.9 (78.4),184.2 (12.8),103.8 (37.1),301.3 (107.5)
DI_Ca_2_6,2787.9 (736.0),234.5 (59.9),198.2 (15.3),113.9 (42.1),325.3 (89.5)


# Regressions

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
# Assuming final_df is your DataFrame
selected_columns = final_df[['Ols_P_0_2', 'H3A_P_0_2', 'M3_P_0_2']]

# Calculate Pearson's correlation matrix
correlation_matrix = selected_columns.corr()

# To obtain R-squared values, square the correlation coefficients
r_squared_matrix = correlation_matrix ** 2

print("Correlation Matrix:")
print(correlation_matrix)

print("\nR-squared Matrix:")
print(r_squared_matrix)

Correlation Matrix:
           Ols_P_0_2  H3A_P_0_2  M3_P_0_2
Ols_P_0_2   1.000000   0.324869  0.908445
H3A_P_0_2   0.324869   1.000000  0.608796
M3_P_0_2    0.908445   0.608796  1.000000

R-squared Matrix:
           Ols_P_0_2  H3A_P_0_2  M3_P_0_2
Ols_P_0_2   1.000000   0.105540  0.825272
H3A_P_0_2   0.105540   1.000000  0.370633
M3_P_0_2    0.825272   0.370633  1.000000
